## Time series to dynamic functional connectivity matrices

In [ ]:
import numpy as np
import glob
from nilearn import datasets
from nilearn import input_data
import pandas as pd
from nilearn.connectome import ConnectivityMeasure
from sklearn.covariance import EmpiricalCovariance

correlation_measure = ConnectivityMeasure(kind='correlation', discard_diagonal=True)
covariance_measure = ConnectivityMeasure(kind='covariance', discard_diagonal=True)
import pickle
import scipy
import scipy.io

### first we load preprocessed files (times series)

Note these files are preprocessed by fmriprep and denoised by xcpengine

In [ ]:
# Change the dir to your data dir
filenames = sorted(glob.glob('/data/analysis/data/time_series/*'))

for f in filenames:
    print(f)
len(filenames)

## We then slice each time series as non-overlapping time windows

each time window has 30 TR (2*30 = 60s)

In [ ]:
pooled_subjects = []
pooled_subjects_win1 = []
pooled_subjects_win2 = []
pooled_subjects_win3 = []
pooled_subjects_win4 = []


for f in filenames:
    # load files
    time_series = np.loadtxt(f, delimiter=',')
    # slicing time series
    time_series_win1 = time_series[0:30,:]
    time_series_win2 = time_series[30:60,:]
    time_series_win3 = time_series[60:90,:]
    time_series_win4 = time_series[90:121,:]
    
    pooled_subjects.append(time_series)
    pooled_subjects_win1.append(time_series_win1)
    pooled_subjects_win2.append(time_series_win2)
    pooled_subjects_win3.append(time_series_win3)
    pooled_subjects_win4.append(time_series_win4)

## We then tranform each time series to a functional connectivity matrix with tools from nilearn

Note that the way nilearn calculate correlation is a little bit different from the normal correlation calculation such as in matlab.

See this [post](https://neurostars.org/t/connectivitymeasure-function-in-nilearn-compare-with-corrcoef-in-matlab/3659/4)

In [ ]:
# calculate correlation matrix and covarance matrix for each sub and each run
correlation_matrices = correlation_measure.fit_transform(pooled_subjects)
correlation_matrices_win1 = correlation_measure.fit_transform(pooled_subjects_win1)
correlation_matrices_win2 = correlation_measure.fit_transform(pooled_subjects_win2)
correlation_matrices_win3 = correlation_measure.fit_transform(pooled_subjects_win3)
correlation_matrices_win4 = correlation_measure.fit_transform(pooled_subjects_win4)


covariance_matricies = covariance_measure.fit_transform(pooled_subjects)
covariance_matricies_win1 = covariance_measure.fit_transform(pooled_subjects_win1)
covariance_matricies_win2 = covariance_measure.fit_transform(pooled_subjects_win2)
covariance_matricies_win3 = covariance_measure.fit_transform(pooled_subjects_win3)
covariance_matricies_win4 = covariance_measure.fit_transform(pooled_subjects_win4)

## Then we can confirm the shape of the transformed correlation matrix

In [ ]:
# Print time series rows should represent how many TRs in your data
# Col should represent how many nodes in your atlas
print(time_series.shape)
print(time_series_win1.shape)
print(time_series_win2.shape)
print(time_series_win3.shape)
print(time_series_win4.shape)

print(correlation_matrices.shape)
print(covariance_matricies.shape)
# Print how many 3-d Images in your data
print(len(pooled_subjects))
# Print how many participants in your data
print(len(pooled_subjects)/3)

## Then we load the meta data 

This step is to classify the matrices into different experimental conditions and participants.

This meta file is a csv file which includes information about the sub index, experimental condition and the file name.

In [ ]:
# Load meta data
meta = pd.read_csv('meta_full.csv')
meta["file"] = meta["file"].str.slice(18, 24, 1) 

In [ ]:
meta

## Then we make a dictionary to include all the functional matrices

Thes dictionary has components, each conponents corresponds to each experimental condition (bore flow frust).
Then it append each matrices into the right position in the dictionary.
Making the dataset as a dictionary is convinient to store as a matlab readale data file (.mat).

In [ ]:
# create a dictionary that includes everything
mats = dict({
            "flow": {"filename": [], 
                     "sub": [], 
                     "run": [], 
                     "time_series": [], 
                     "corr_mats": [], 
                     "win_1": [],
                     "win_2": [],
                     "win_3": [],
                     "win_4": [],
                     "cov_mats": [],
                     "cov_win_1": [],
                     "cov_win_2": [],
                     "cov_win_3": [],
                     "cov_win_4": []}, 
            "bore": {"filename": [], 
                     "sub": [], 
                     "run": [], 
                     "time_series": [], 
                     "corr_mats": [], 
                     "win_1": [],
                     "win_2": [],
                     "win_3": [],
                     "win_4": [],
                     "cov_mats": [],
                     "cov_win_1": [],
                     "cov_win_2": [],
                     "cov_win_3": [],
                     "cov_win_4": []}, 
            "frus": {"filename": [], 
                     "sub": [], 
                     "run": [], 
                     "time_series": [], 
                     "corr_mats": [], 
                     "win_1": [],
                     "win_2": [],
                     "win_3": [],
                     "win_4": [],
                     "cov_mats": [],
                     "cov_win_1": [],
                     "cov_win_2": [],
                     "cov_win_3": [],
                     "cov_win_4": []}
            })


for i in range(len(filenames)):
    for j in range(len(meta)):
        if meta.iloc[j]['sub'] in filenames[i] and meta.iloc[j]['file'] in filenames[i]:
            if meta.iloc[j]['condition'] == 'flow':
                print('filename is', i, "flow")
                print(meta.iloc[j]['sub'], meta.iloc[j]['file'])
                
                # append time series and corr-mats, cov-mats into the data
                mats["flow"]["time_series"].append(pooled_subjects[i])
                mats["flow"]["corr_mats"].append(correlation_matrices[i])
                mats["flow"]["win_1"].append(correlation_matrices_win1[i])
                mats["flow"]["win_2"].append(correlation_matrices_win2[i])
                mats["flow"]["win_3"].append(correlation_matrices_win3[i])
                mats["flow"]["win_4"].append(correlation_matrices_win4[i])
                
                mats["flow"]["cov_mats"].append(covariance_matricies[i])
                mats["flow"]["cov_win_1"].append(covariance_matricies_win1[i])
                mats["flow"]["cov_win_2"].append(covariance_matricies_win2[i])
                mats["flow"]["cov_win_3"].append(covariance_matricies_win3[i])
                mats["flow"]["cov_win_4"].append(covariance_matricies_win4[i])
                
                # append metadata information into the data
                mats["flow"]["filename"].append(filenames[i])
                mats["flow"]["sub"].append(meta.iloc[j]['sub'])
                mats["flow"]["run"].append(meta.iloc[j]['file'])
            elif meta.iloc[j]['condition'] == 'boredom':
                print('filename is', i, "bore")
                print(meta.iloc[j]['sub'], meta.iloc[j]['file'])
                
                # append time series and corr-mats, cov-mats into the data
                mats["bore"]["time_series"].append(pooled_subjects[i])
                mats["bore"]["corr_mats"].append(correlation_matrices[i])
                mats["bore"]["win_1"].append(correlation_matrices_win1[i])
                mats["bore"]["win_2"].append(correlation_matrices_win2[i])
                mats["bore"]["win_3"].append(correlation_matrices_win3[i])
                mats["bore"]["win_4"].append(correlation_matrices_win4[i])
                
                mats["bore"]["cov_mats"].append(covariance_matricies[i])
                mats["bore"]["cov_win_1"].append(covariance_matricies_win1[i])
                mats["bore"]["cov_win_2"].append(covariance_matricies_win2[i])
                mats["bore"]["cov_win_3"].append(covariance_matricies_win3[i])
                mats["bore"]["cov_win_4"].append(covariance_matricies_win4[i])
                
                # append metadata information into the data
                mats["bore"]["filename"].append(filenames[i])
                mats["bore"]["sub"].append(meta.iloc[j]['sub'])
                mats["bore"]["run"].append(meta.iloc[j]['file'])
            elif meta.iloc[j]['condition'] == 'frustration':
                print('filename is', i, "frus")
                print(meta.iloc[j]['sub'], meta.iloc[j]['file'])
                
                # append time series and corr-mats, cov-mats into the data
                mats["frus"]["time_series"].append(pooled_subjects[i])
                mats["frus"]["corr_mats"].append(correlation_matrices[i])
                mats["frus"]["win_1"].append(correlation_matrices_win1[i])
                mats["frus"]["win_2"].append(correlation_matrices_win2[i])
                mats["frus"]["win_3"].append(correlation_matrices_win3[i])
                mats["frus"]["win_4"].append(correlation_matrices_win4[i])
                
                mats["frus"]["cov_mats"].append(covariance_matricies[i])
                mats["frus"]["cov_win_1"].append(covariance_matricies_win1[i])
                mats["frus"]["cov_win_2"].append(covariance_matricies_win2[i])
                mats["frus"]["cov_win_3"].append(covariance_matricies_win3[i])
                mats["frus"]["cov_win_4"].append(covariance_matricies_win4[i])
                
                # append metadata information into the data
                mats["frus"]["filename"].append(filenames[i])
                mats["frus"]["sub"].append(meta.iloc[j]['sub'])
                mats["frus"]["run"].append(meta.iloc[j]['file'])

## Then we can save the file as .mat file

.mat can make the file matlab readable. 

In [ ]:
# Save file to pickle
# with open('output_0108_nogsr.pickle', 'wb') as handle:
#    pickle.dump(mats, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Save file to .mat (this is preferred because it is readable by matlab)
scipy.io.savemat('/data/output.mat', mats)

In [ ]:
import matplotlib.pyplot as plt
print(time_series.shape)
plt.plot(pooled_subjects[2])
plt.title('Extracted Time Series from Power atlas')
plt.xlabel('Scan number')
plt.ylabel('BOLD signal')
plt.tight_layout()
plt.savefig('figures/method/extracted_time_series.png')